In [512]:
import csv
import re
import pandas as pd
import random
import copy
import nltk

NAME = "fleep"
GENDER = "M"
CLASS = "hunter"
RACE = "dwarf"

quests = pd.read_csv("quests.csv")
#print(list(quests.columns.values))
details = quests.Details
rawStr = ""
for d in details:
    rawStr += str(d)

In [519]:
prepositions = ["were","of","the","from","is","on","when","at","for",
               "are", "to", "we", "a", "who", "i", "he", "she", "in",
               "her","his","their","they", "but", "however", "therefore",
               "that", "has", "with"] #words we don't want to end a sentence with
def get_ran_word(cfdist,word,myinterest):
    if myinterest in cfdist[word].keys() and random.random() > 0.1 and myinterest is not word:
        return myinterest
    if "." in cfdist[word].keys() and random.random() > 0.5 and word not in prepositions:
        return "."
    d = copy.copy(cfdist[word])
    lend = len(d)
    if lend == 0:
        return random.choice(["the","of"])
    if lend > 10:
        sampleNum = lend // 5; #sample from the top 20% words
    else:
        sampleNum = lend
    words = []
    for i in range(sampleNum):
        if(lend == 1):
            words.append(d.keys()[0])
            break;
        words.append(d.max());
        del(d[words[i]]);
    return random.choice(words)

def generate_ran_sentence(cfdist, initwords, interest="", minwords=5, maxwords = 15):#, num=15):
    i = 1
    interestCount = 0
    foundInterest = False
    words = [initwords.capitalize()]
    word = get_ran_word(cfdist,initwords.split(" ")[-1],interest)
    while word is not "." and i < maxwords:
        words.append(word)
        if foundInterest: #it has become uninteresting...
            word = get_ran_word(cfdist,word,"")
        else:
            word = get_ran_word(cfdist,word,interest)
        if word is interest:
            foundInterest = True
    if not foundInterest or len(words) < minwords: #keep going til you get an interest
        return generate_ran_sentence(cfdist,initwords,interest=interest,minwords=minwords,maxwords=maxwords)
    return words
        
def format_simple(document):
    doc = document.lower()                                           #convert to lower case
    doc = re.sub(r'[\?\{\}\[\]\|\(\)!,:\-;\$\"<>]',' ',doc)           #remove punctuations
    doc = re.sub(r'[\']','',doc)                                     # remove appostrophes
    doc = re.sub(r'\.', ' . ', doc)                                  #add spaces between periods
    doc = re.sub(r'[\s]+',' ', doc)                                  #remove whitespace clumps
    doc =  doc.strip()                                               #remove trailing whitespace
    return doc

def replace_variables(document):
    doc = document.lower()
    if GENDER == "M":
        doc = re.sub(r'\$g(.*):(.*);',r'\g<1>',doc)
    else:
        doc = re.sub(r'\$g(.*):(.*);',r'\g<2>',doc)
    doc = re.sub(r'\$c',CLASS,doc)
    doc = re.sub(r'\$n',NAME,doc)
    doc = re.sub(r'\$r',RACE,doc)
    return doc
    
def format_simple_keep_punctuation(document):
    doc = document.lower()                                           #convert to lower case
    doc = re.sub(r'[\[\]\|<>\-]','',doc)                                  #remove unneeded punctuations
    doc = re.sub(r'([\.\?\{\}\[\]\(\)!,:\-;\$\"\'])',r' \g<1> ',doc)           #put spaces between tagged punctuations
    doc = re.sub(r'[\s]+',' ', doc)                                  #remove whitespace clumps
    doc =  doc.strip()                                               #remove trailing whitespace
    return doc

In [514]:
# rawStr = replace_variables(rawStr)
# rawStr = format_simple_keep_punctuation(rawStr)
# tokens = rawStr.split(' ')
# text = nltk.Text(tokens)

alldeets = "".join(str(d) for d in details)
alldeets = replace_variables(alldeets)
#construct text_nopunct
rawStr = format_simple(alldeets)
tokens_nopunct = rawStr.split(' ')
text_nopunct = nltk.Text(tokens_nopunct)
#construct text (with punctuation)
rawStr = format_simple_keep_punctuation(alldeets)
tokens = rawStr.split(' ')
text = nltk.Text(tokens_nopunct)

# get cfds from bigrams
bigrams_nopunct = nltk.bigrams(text_nopunct)
cfd_nopunct = nltk.ConditionalFreqDist(bigrams_nopunct)

bigrams = nltk.bigrams(text)
cfd = nltk.ConditionalFreqDist(bigrams)

In [515]:
postag = nltk.pos_tag(text)

In [305]:
tag_fd = nltk.FreqDist(tag for (word, tag) in postag)
print(tag_fd.most_common())
word_fd = nltk.FreqDist(word for (word, tag) in postag)
#print(word_fd.most_common())

[('NN', 100889), ('IN', 63764), ('DT', 57068), ('VB', 43443), ('.', 41763), ('JJ', 39215), ('PRP', 37256), ('NNS', 31305), ('RB', 26023), ('VBP', 21146), ('TO', 21129), ('CC', 15984), ('PRP$', 15421), ('VBZ', 13518), ('VBN', 12683), ('MD', 11906), ('VBD', 9451), ('VBG', 8848), ('RP', 3152), ('CD', 2746), ('WRB', 2373), ('WP', 2208), ('WDT', 1943), ('JJR', 1556), ('JJS', 1439), (':', 1180), ('EX', 1123), ('RBR', 645), ('PDT', 484), ('RBS', 322), ('FW', 109), ('UH', 95), ('NNP', 57), ('SYM', 55), ('WP$', 16), ('$', 3), ("''", 2), ('NNPS', 2)]


In [549]:
w = "the"

print(cfd[w])
freqs = cfd[w]
print(str(float((201/33449))))
print(freqs.most_common()[:100])

<FreqDist with 5646 samples and 33449 outcomes>
0.0
[('other', 201), ('area', 186), ('time', 180), ('road', 174), ('ruins', 174), ('first', 163), ('only', 161), ('burning', 155), ('last', 153), ('most', 152), ('dark', 139), ('barrens', 124), ('city', 123), ('blood', 123), ('world', 115), ('spirits', 115), ('key', 113), ('land', 112), ('path', 110), ('valley', 107), ('power', 106), ('way', 103), ('battle', 103), ('ground', 103), ('light', 103), ('argent', 100), ('one', 99), ('cenarion', 99), ('great', 98), ('dead', 97), ('same', 97), ('best', 96), ('crystal', 95), ('scarlet', 95), ('rest', 94), ('water', 93), ('temple', 91), ('forest', 90), ('name', 86), ('heart', 83), ('legion', 82), ('coast', 80), ('fel', 79), ('book', 77), ('creatures', 76), ('next', 74), ('undercity', 74), ('old', 72), ('tournament', 71), ('village', 70), ('mine', 69), ('black', 69), ('end', 68), ('field', 67), ('plague', 67), ('enemy', 66), ('town', 66), ('iron', 64), ('earth', 64), ('vrykul', 64), ('top', 63), ('s

In [530]:
#uses bigram frequencies to generate random sentences of a common theme
def get_paragraph(startwords,interest):
    numsentences = len(startwords)
    interests = [interest]*numsentences
    sentences = []
    randsents = []
    for j,k in zip(startwords,interests):
        randsents.append(generate_ran_sentence(cfd_nopunct, j,interest=k,maxwords=20))
    for sents in randsents:
        sentences.append(" ".join(sents))
    return (". ".join(sentences) + ".")
        
startwords = ["the humans","the orcs", "the world is","a new beginning", "return to the"]
interests = ["stormwind","harpy","water"]
for interest in interests:
    print(get_paragraph(startwords,interest))
    print("")

The humans have spoken in stormwind. The orcs and realize that stormwind. The world is entitled to stormwind keep. A new beginning to sponsor you into stormwind guard. Return to the stormwind is hard to stormwind.

The humans go and valor and worse the harpy named iverron as these blood. The orcs emerging from southern duskwood as he carries the harpy wenches to report from which. The world is causing the harpy named in dire. A new beginning ive a harpy wenches will all levels at scourgeholme now its energy. Return to the harpy named sidragos which the harpy nesting.

The humans have water supply our hunters of twenty. The orcs of water elementals at him return. The world is water well it with water. A new beginning and water on da south through them. Return to the water on bloodmyst when the water.



In [550]:
from nltk import PCFG
from nltk.parse.generate import generate, demo_grammar
from nltk import CFG
from operator import itemgetter
from random import shuffle

nonterms = {".",",",r"PRP$",":","$","\'\'", "(", ")", "CC", "CD", "DT", "EX", "FW", "IN", r"WP$"}
nonpos = {"i", "\"", "\'"}
termvalues = {}
#test = {"NN","JJ","JJR","MD","RB","VB","RBS","RBR","UH", "VBP", "VBZ", "WDT", "CC", "NNS", "PRP", "VBN", "EX"}

limit = 30
postags = postag
shuffle(postags)
postags.sort(key=itemgetter(1))
my_grammar = """
  S -> KI NK
  S -> COLCD N
  S -> TR NT
  S -> TK
  COLCD -> COL CD
  TK -> RT TO AND TAL TO NNP
  TK -> TR TO AND TAL TO NNP
  NK -> CD NN AT NNP
  NK -> NNP THE NNP
  NT -> TO NNP
  NT -> TO NNP AND KI CD NN
  NT -> TO NNP AND COL CD NN
  NT -> TO NNP AND KI NNP THE NNP
  N -> JJ NN FR NNP
  N -> JJ NN AT NNP
  N -> JJ NN
  N -> NN
  COL -> 'Collect' | 'Gather' | 'Find'
  FR -> 'from'
  KI -> 'Kill' | 'Slay'
  TR -> 'Travel' | 'Journey'
  AT -> 'At' | 'Located In'
  THE -> 'The'
  OF -> 'Of'
  AND -> 'And'
  RT -> 'Return'
  TAL -> 'Talk'
"""

my_grammar += "  " + "\n  ".join([(r"CD -> '" + str(num) + r"'") for num in [1,5,10]])
for (a,b) in postags:
    if b not in nonterms and a not in nonpos and len(a)>1:
        if termvalues.has_key(b) == False:
            termvalues[b] = 1
            my_grammar += "\n  " + "\n  ".join([(b + r" -> '" + a + r"'")])
        elif termvalues.get(b) < limit:
            termvalues[b] += 1
            my_grammar += "\n  " + "\n  ".join([(b + r" -> '" + a + r"'")]) 

grammar = CFG.fromstring(my_grammar)
fulllist = []
for sentence in generate(grammar):
    fulllist.append(sentence)

ranlist = [None]*10
for i in range(0,10):
    ranlist[i] = random.choice(fulllist)

for sentence in ranlist:
    print(' '.join(sentence))


Travel to moonwell And Kill zulmarosh The zulaman
Journey to khan And Kill 5 compound
Travel to jabber And Slay xoroth The zin
Travel to zin And Slay youve The zin
Gather 5 diet arakkoa Located In zulmarosh
Find 1 large duty Located In yer
Journey to kreldig And Slay xoroth The jibber
Find 10 large dem At youve
Journey to zulmarosh And Slay 1 energy
Travel to korkron And Slay youve The khan
